%md
## Informações Gerais
 | Informações | Detalhes |
 |------------|-------------|
 |Nome Tabela | silver.psa |
 |Origem | bronze.dados_clientes / bronze.notas_fiscais /bronze.analises_tributarias/ bronze.tarefas_projetos / bronze.logs_sistema |

## Histórico de Atualizações
 | Data | Desenvolvido por | Motivo |
 |:----:|--------------|--------|
 |27/10/2025 | Kálita Boni | Criação do notebook |


In [0]:
%sql
-- 1) Fixar o catálogo
USE CATALOG psa;

-- 2) Criar/usar o schema (sem repetir o catálogo aqui)
CREATE SCHEMA IF NOT EXISTS psa_curated;
USE SCHEMA psa_curated;

-- 3) Conferir
SELECT current_catalog() AS catalog, current_schema() AS schema;

-- (opcional) ver todos os schemas do catálogo
SHOW SCHEMAS IN psa;



databaseName
bronze
default
gold
information_schema
psa_curated
silver


In [0]:
%sql
CREATE OR REPLACE TABLE psa_curated.notas_fiscais AS
WITH bronze AS (
  SELECT
    TRIM(NumeroNota)                                   AS numero_nota,
    CAST(TO_DATE(TO_TIMESTAMP(DataEmissao)) AS DATE)   AS data_emissao,
    TRIM(ClienteID)                                    AS cliente_id,
    CAST(ValorTotal AS DECIMAL(18,2))                  AS valor_total,
    Impostos                                           AS impostos_raw,  -- STRING
    Itens                                              AS itens_raw      -- pode ser STRING/JSON
  FROM psa.bronze.notas_fiscais
  WHERE TRIM(NumeroNota) IS NOT NULL
),

/* tenta ler Impostos como JSON; se não for JSON, vai dar NULL (e tratamos depois) */
json_try AS (
  SELECT
    numero_nota, data_emissao, cliente_id, valor_total, itens_raw, impostos_raw,
    CAST(get_json_object(impostos_raw, '$.ISS')    AS DECIMAL(18,2)) AS iss_json,
    CAST(get_json_object(impostos_raw, '$.PIS')    AS DECIMAL(18,2)) AS pis_json,
    CAST(get_json_object(impostos_raw, '$.COFINS') AS DECIMAL(18,2)) AS cofins_json
  FROM bronze
),

/* fallback por REGEX quando Impostos não for JSON (ex.: XML/texto) */
impostos_ok AS (
  SELECT
    numero_nota, data_emissao, cliente_id, valor_total, itens_raw, impostos_raw,
    COALESCE(
      iss_json,
      CAST(REPLACE(REGEXP_EXTRACT(impostos_raw, '(?i)ISS[^0-9]*([0-9]+[\\.,]?[0-9]*)', 1), ',', '.') AS DECIMAL(18,2))
    ) AS iss,
    COALESCE(
      pis_json,
      CAST(REPLACE(REGEXP_EXTRACT(impostos_raw, '(?i)PIS[^0-9]*([0-9]+[\\.,]?[0-9]*)', 1), ',', '.') AS DECIMAL(18,2))
    ) AS pis,
    COALESCE(
      cofins_json,
      CAST(REPLACE(REGEXP_EXTRACT(impostos_raw, '(?i)COFINS[^0-9]*([0-9]+[\\.,]?[0-9]*)', 1), ',', '.') AS DECIMAL(18,2))
    ) AS cofins
  FROM json_try
),

curated AS (
  SELECT
    numero_nota,
    data_emissao,
    cliente_id,
    COALESCE(valor_total, CAST(0 AS DECIMAL(18,2))) AS valor_total,
    named_struct(
      'iss',   COALESCE(iss,    CAST(0 AS DECIMAL(18,2))),
      'pis',   COALESCE(pis,    CAST(0 AS DECIMAL(18,2))),
      'cofins',COALESCE(cofins, CAST(0 AS DECIMAL(18,2)))
    ) AS impostos_detalhados,
    itens_raw AS itens
  FROM impostos_ok
),

-- remove duplicatas por número da nota (mantém a mais recente por data)
dedup AS (
  SELECT * EXCEPT(rn) FROM (
    SELECT c.*,
           ROW_NUMBER() OVER (PARTITION BY numero_nota ORDER BY data_emissao DESC NULLS LAST) rn
    FROM curated c
  ) WHERE rn = 1
)

SELECT
  numero_nota,
  data_emissao,
  valor_total,
  impostos_detalhados,
  cliente_id,
  itens
FROM dedup;



num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE OR REPLACE TABLE psa_curated.notas_fiscais_itens AS
WITH base AS (
  SELECT
    TRIM(NumeroNota)                                   AS numero_nota,
    CAST(TO_DATE(TO_TIMESTAMP(DataEmissao)) AS DATE)   AS data_emissao,
    TRIM(ClienteID)                                    AS cliente_id,
    Itens                                              AS itens_raw
  FROM psa.bronze.notas_fiscais
  WHERE TRIM(NumeroNota) IS NOT NULL
),

parsed AS (
  SELECT
    numero_nota, data_emissao, cliente_id,
    FROM_JSON(
      itens_raw,
      'STRUCT<Item:ARRAY<STRUCT<Descricao:STRING,Quantidade:INT,ValorUnitario:DECIMAL(18,2)>>>'
    ) AS itens_json
  FROM base
),

exploded AS (
  SELECT
    numero_nota, data_emissao, cliente_id,
    EXPLODE(itens_json.Item) AS item
  FROM parsed
  WHERE itens_json IS NOT NULL
)

SELECT
  numero_nota,
  data_emissao,
  cliente_id,
  TRIM(item.Descricao)                                    AS descricao_item,
  COALESCE(CAST(item.Quantidade AS INT), 0)               AS quantidade,
  COALESCE(CAST(item.ValorUnitario AS DECIMAL(18,2)), CAST(0 AS DECIMAL(18,2))) AS valor_unitario,
  COALESCE(CAST(item.Quantidade AS INT), 0) *
  COALESCE(CAST(item.ValorUnitario AS DECIMAL(18,2)), CAST(0 AS DECIMAL(18,2))) AS valor_total_item
FROM exploded;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- 0. Quais colunas existem e tipos?
DESCRIBE TABLE psa.bronze.notas_fiscais;

-- 1. Total na Bronze
SELECT COUNT(*) FROM psa.bronze.notas_fiscais;

-- 2. Quantas têm NumeroNota preenchido?
SELECT COUNT(*) 
FROM psa.bronze.notas_fiscais 
WHERE TRIM(NumeroNota) IS NOT NULL;

-- 3. Prévia simples da Bronze (confirme nomes exatamente iguais)
SELECT NumeroNota, DataEmissao, ClienteID, ValorTotal, Impostos, Itens
FROM psa.bronze.notas_fiscais
LIMIT 5;


NumeroNota,DataEmissao,ClienteID,ValorTotal,Impostos,Itens
NF-5839/2024,2024-09-06,CLI0033,34598.52,1592.2 206.99 955.32,Planejamento Tributário Anual 4 15922.0 Planejamento Tributário Anual 3 15922.0
NF-6154/2024,2024-10-03,CLI0006,36120.57,1662.24 216.09 997.35,Consultoria ISS Serviços 7 33244.89
NF-8423/2024,2024-09-19,CLI0026,10020.1,461.12 59.95 276.67,Auditoria Fiscal Completa 2 4611.18 Consultoria ISS Serviços 10 4611.18
NF-9331/2024,2024-02-04,CLI0039,14188.99,652.97 84.89 391.78,Serviços de Consultoria Tributária 2 6529.68 Planejamento Tributário Anual 7 6529.68
NF-2592/2024,2024-07-22,CLI0001,31505.24,1449.85 188.48 869.91,Revisão Declarações Fiscais 1 14498.5 Análise Fiscal PIS/COFINS 7 14498.5


In [0]:
%sql
SELECT current_catalog(), current_schema();


current_catalog(),current_schema()
workspace,default
